In [1]:
# This section is to move to the directory on Google Drive
import os
# os.chdir('drive/MyDrive/ComVis_20211')

In [2]:
import cv2
import numpy as np
from tqdm import tqdm

In [3]:
# Change this section to your dataset
path = "Parasyte"

In [4]:
# sort rectangles by their area
def sort_rect(rect):
    return rect[4]

In [5]:
def detect_overlap(rect1, rect2):
    x1, y1, w1, h1 = rect1[0], rect1[1], rect1[2], rect1[3]
    x2, y2, w2, h2 = rect2[0], rect2[1], rect2[2], rect2[3]

    #check if rect 2 inside rect 1
    if ((x2+w2) <= (x1+w1)) and (x2 >= x1) and ((y2+h2) <= (y1+h1)) and (y2 >= y1):
        return True

    #check percentage of iou for rect 2 and rect 1
    xa = max(x1,x2)
    ya = max(y1,y2)
    xb = min(x1+w1, x2+w2)
    yb = min(y1+h1, y2+h2)

    interArea = max(0, xb-xa+1)*max(0,yb-ya+1)

    rect1Area = w1*h1
    rect2Area = w2*h2

    iou = interArea/(rect1Area + rect2Area - interArea)
    if iou > 0.2:
        return True
    return False

In [6]:
# class to get the total overlapped area of multiple rectangles
class area:
    def __init__(self, x, y, w, h):
        self.x1, self.y1 = x, y
        self.x2, self.y2 = x + w, y + h
        self.span = w * h
        self.filled = []
    def cutout(a1, a2):
        if a1.x2 >= a2.x1 and a2.x2 >= a1.x1 and a1.y2 >= a2.y1 and a2.y2 >= a1.y1:
            x1 = max(a1.x1, a2.x1)
            y1 = max(a1.y1, a2.y1)
            overlap = area(x1, y1, min(a1.x2, a2.x2) - x1, min(a1.y2, a2.y2) - y1)
            for a3 in a1.filled:
                a3.cutout(overlap)
            a1.filled.append(overlap)
    def area(self):
        return self.span - sum(a.area() for a in self.filled)

In [7]:
def coverage_area(dir):
    summ = 0
    img_list = os.listdir(dir)
    for img_name in img_list:
        img = os.path.join(chapter_path, img_name)
        # Load image, grayscale, Gaussian blur, Canny edge detection
        image = cv2.imread(img)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (3,3), 0)
        canny = cv2.Canny(blurred, 100, 200, 1)

        # Find contours
        cnts = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        # cnts, _ = contours.sort_contours(cnts, method="left-to-right")
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)

        (img_x, img_y, img_c) = image.shape
        threshold_lower = img_x*img_y/16
        vertices = []
        for i in range(len(cnts)):
            c = cnts[i]
            # Obtain bounding rectangle for each contour
            x,y,w,h = cv2.boundingRect(c)

            if ((w*h) > threshold_lower):
                vertices.append((x,y,w,h, w*h))
                # Find ROI of the contour

        vertices = sorted(vertices, key=sort_rect, reverse=True)
        overlapped = np.array([0]*len(vertices))
        for i in range(len(vertices)):
            rect1 = vertices[i]
            for j in range(i+1, len(vertices)):
                rect2 = vertices[j]
                if detect_overlap(rect1, rect2):
                    overlapped[j] = 1

        true_bound = []
        for i in range(len(vertices)):
            if overlapped[i] == 0:
                true_bound.append(vertices[i])

        rect_ls = []
        for rect in true_bound:
            r = area(rect[0], rect[1], rect[2], rect[3])
            rect_ls.append(r)
      
        processed = []
        for r in rect_ls:
            for r_p in processed:
                r_p.cutout(r)
            processed.append(r)
        summ += sum(r.area() for r in processed)/(img_x*img_y)
    return summ/len(img_list)

In [8]:
chapters = os.listdir(path)
coverage = []
for chapter in tqdm(chapters):
    chapter_path = os.path.join(path, chapter)
    coverage.append(coverage_area(chapter_path))

100%|██████████| 64/64 [22:20<00:00, 20.95s/it]


In [9]:
print(coverage)

[0.5792213259991988, 0.6743565463362069, 0.6873314063435485, 0.6929141998063747, 0.6947069048392776, 0.7039816106320798, 0.6448881696428572, 0.7119884177826985, 0.7455119898719095, 0.7081287063827426, 0.6842257696467776, 0.7288880801776141, 0.6258199073636589, 0.7270898386164117, 0.6670822845162457, 0.6137769366224421, 0.6409148297991071, 0.6836618397535394, 0.7367774076169734, 0.6852972907258451, 0.66120799821269, 0.6739871401684404, 0.6790518962414216, 0.6843762889255516, 0.6286343319882687, 0.6864122542448615, 0.7271361706881144, 0.6884189148606547, 0.6505370866845396, 0.716014697834547, 0.6930017966190052, 0.679211763392857, 0.6844143842233653, 0.6991985524102402, 0.7014290413067221, 0.7102260493465148, 0.6903681429406077, 0.6880495525686545, 0.6658265466812356, 0.655126142563336, 0.6849604487823951, 0.6474586684539769, 0.6760659656035424, 0.6639803178681364, 0.6449804246945486, 0.6222356397601672, 0.6843048053671872, 0.7016038312369497, 0.6937139023972775, 0.6684275290185026, 0.68